라이브러리

In [1]:
import pandas as pd
import numpy as np
from keras.utils import *
from keras.layers import LSTM, Dense, TimeDistributed
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import *
from keras.losses import Huber
from sklearn.preprocessing import RobustScaler

model_name = 'model/Aqua_all_robust'

In [3]:
japonica_training = pd.DataFrame()
japonica_validation = pd.DataFrame()
for i in range(5):
    tank = i + 1
    japonica_training_food_supply_tb = pd.read_csv("dataset/japonica_training_food_supply_tb.csv")
    japonica_training_sensor_val_tb = pd.read_csv("dataset/japonica_training_sensor_val_tb.csv")
    japonica_validation_food_supply_tb = pd.read_csv("dataset/japonica_validation_food_supply_tb.csv")
    japonica_validation_sensor_val_tb = pd.read_csv("dataset/japonica_validation_sensor_val_tb.csv")
    japonica_training_food_supply_tb = japonica_training_food_supply_tb[japonica_training_food_supply_tb['tank_id']==tank]
    japonica_validation_food_supply_tb = japonica_validation_food_supply_tb[japonica_validation_food_supply_tb['tank_id']==tank]
    japonica_training_food_supply_tb = japonica_training_food_supply_tb[japonica_training_food_supply_tb['feed_quantity'].notnull()]
    japonica_validation_food_supply_tb = japonica_validation_food_supply_tb[japonica_validation_food_supply_tb['feed_quantity'].notnull()]
    japonica_training_food_supply_tb['feed_dt'] = pd.to_datetime(japonica_training_food_supply_tb['feed_dt'], format='%Y%m%d%H%M', errors='raise')
    japonica_training_food_supply_tb["feed_dt"] = japonica_training_food_supply_tb["feed_dt"].apply(str)
    japonica_training_food_supply_tb["feed_dt"] = japonica_training_food_supply_tb["feed_dt"].str.slice(start=0, stop=16)
    japonica_validation_food_supply_tb['feed_dt'] = pd.to_datetime(japonica_validation_food_supply_tb['feed_dt'], format='%Y%m%d%H%M', errors='raise')
    japonica_validation_food_supply_tb["feed_dt"] = japonica_validation_food_supply_tb["feed_dt"].apply(str)
    japonica_validation_food_supply_tb["feed_dt"] = japonica_validation_food_supply_tb["feed_dt"].str.slice(start=0, stop=16)
    japonica_training_sensor_val_tb = japonica_training_sensor_val_tb[japonica_training_sensor_val_tb['tank_id']==tank]
    japonica_validation_sensor_val_tb = japonica_validation_sensor_val_tb[japonica_validation_sensor_val_tb['tank_id']==tank]
    japonica_training_sensor_val_tb["mea_dt"] = japonica_training_sensor_val_tb["mea_dt"].apply(str)
    japonica_training_sensor_val_tb["mea_dt"] = japonica_training_sensor_val_tb["mea_dt"].str.slice(start=0, stop=16)
    japonica_training_sensor_val_tb['mea_dt'] = pd.to_datetime(japonica_training_sensor_val_tb['mea_dt'], format='%Y-%m-%d %H:%M', errors='raise')
    japonica_validation_sensor_val_tb["mea_dt"] = japonica_validation_sensor_val_tb["mea_dt"].apply(str)
    japonica_validation_sensor_val_tb["mea_dt"] = japonica_validation_sensor_val_tb["mea_dt"].str.slice(start=0, stop=16)
    japonica_validation_sensor_val_tb['mea_dt'] = pd.to_datetime(japonica_validation_sensor_val_tb['mea_dt'], format='%Y-%m-%d %H:%M', errors='raise')
    japonica_training_sensor_val_tb['date'] = pd.to_datetime(japonica_training_sensor_val_tb['mea_dt']).dt.date
    japonica_training_food_supply_tb['date'] = pd.to_datetime(japonica_training_food_supply_tb['feed_dt']).dt.date
    common_dates = japonica_training_sensor_val_tb['date'].isin(japonica_training_food_supply_tb['date'])
    japonica_training_sensor_val_tb = japonica_training_sensor_val_tb[common_dates]
    japonica_training_sensor_val_tb = japonica_training_sensor_val_tb.drop('date', axis=1)
    japonica_validation_sensor_val_tb['date'] = pd.to_datetime(japonica_validation_sensor_val_tb['mea_dt']).dt.date
    japonica_validation_food_supply_tb['date'] = pd.to_datetime(japonica_validation_food_supply_tb['feed_dt']).dt.date
    common_dates = japonica_validation_sensor_val_tb['date'].isin(japonica_validation_food_supply_tb['date'])
    japonica_validation_sensor_val_tb = japonica_validation_sensor_val_tb[common_dates]
    japonica_validation_sensor_val_tb = japonica_validation_sensor_val_tb.drop('date', axis=1)
    japonica_training_sensor_val_tb.set_index('mea_dt', inplace=True)
    japonica_training_sensor_val_tb = japonica_training_sensor_val_tb.resample('30min').mean()
    japonica_training_sensor_val_tb = japonica_training_sensor_val_tb.reset_index()
    japonica_training_sensor_val_tb = japonica_training_sensor_val_tb.dropna()
    japonica_validation_sensor_val_tb.set_index('mea_dt', inplace=True)
    japonica_validation_sensor_val_tb = japonica_validation_sensor_val_tb.resample('30min').mean()
    japonica_validation_sensor_val_tb = japonica_validation_sensor_val_tb.reset_index()
    japonica_validation_sensor_val_tb = japonica_validation_sensor_val_tb.dropna()
    training_counts = japonica_training_sensor_val_tb.groupby(japonica_training_sensor_val_tb['mea_dt'].dt.date).size()
    validation_counts = japonica_validation_sensor_val_tb.groupby(japonica_validation_sensor_val_tb['mea_dt'].dt.date).size()
    to_delete = training_counts[training_counts != 48].index
    japonica_training_sensor_val_tb = japonica_training_sensor_val_tb[~japonica_training_sensor_val_tb['mea_dt'].dt.date.isin(to_delete)]
    to_delete = validation_counts[validation_counts != 48].index
    japonica_validation_sensor_val_tb = japonica_validation_sensor_val_tb[~japonica_validation_sensor_val_tb['mea_dt'].dt.date.isin(to_delete)]
    japonica_training_sensor_val_tb["mea_dt"] = japonica_training_sensor_val_tb["mea_dt"].dt.strftime('%Y-%m-%d %H:%M')
    japonica_training_sensor_val_tb["mea_dt"] = japonica_training_sensor_val_tb["mea_dt"].str.slice(start=0, stop=16)
    japonica_validation_sensor_val_tb["mea_dt"] = japonica_validation_sensor_val_tb["mea_dt"].dt.strftime('%Y-%m-%d %H:%M')
    japonica_validation_sensor_val_tb["mea_dt"] = japonica_validation_sensor_val_tb["mea_dt"].str.slice(start=0, stop=16)
    japonica_training_features = pd.merge(left = japonica_training_sensor_val_tb, right = japonica_training_food_supply_tb, how = "left", left_on = ["farm_id","tank_id", "mea_dt"], right_on = ["farm_id","tank_id", "feed_dt"])
    japonica_validation_features = pd.merge(left = japonica_validation_sensor_val_tb, right = japonica_validation_food_supply_tb, how = "left", left_on = ["farm_id","tank_id", "mea_dt"], right_on = ["farm_id","tank_id", "feed_dt"])
    japonica_training_features['mea_dt'] = pd.to_datetime(japonica_training_features['mea_dt'], format='%Y-%m-%d %H:%M', errors='raise')
    japonica_validation_features['mea_dt'] = pd.to_datetime(japonica_validation_features['mea_dt'], format='%Y-%m-%d %H:%M', errors='raise')
    feature_origin = ['tank_id','mea_dt', 'do_mg','do_temp', 'ph', 'orp', 'co2_mg', 'air_oxy', 'light_ma', 'feed_quantity', 'water_quantity']
    japonica_training_features = japonica_training_features[feature_origin]
    japonica_validation_features = japonica_validation_features[feature_origin]
    japonica_training_features = japonica_training_features.fillna(0)
    japonica_validation_features = japonica_validation_features.fillna(0)
    japonica_training_features.set_index('mea_dt', inplace=True)
    japonica_validation_features.set_index('mea_dt', inplace=True)
    japonica_training_data = japonica_training_features.sort_index()
    japonica_validation_data = japonica_validation_features.sort_index()
    japonica_training_data = japonica_training_data.reset_index()
    japonica_validation_data = japonica_validation_data.reset_index()
    japonica_training_data = japonica_training_data[['do_mg','do_temp', 'ph', 'orp', 'co2_mg', 'feed_quantity', 'water_quantity']]
    japonica_validation_data = japonica_validation_data[['do_mg','do_temp', 'ph', 'orp', 'co2_mg', 'feed_quantity', 'water_quantity']]
    japonica_training = pd.concat([japonica_training, japonica_training_data])
    japonica_validation = pd.concat([japonica_validation, japonica_validation_data])
    japonica_training.reset_index(drop=True, inplace=True)
    japonica_validation.reset_index(drop=True, inplace=True)

In [4]:
scaler = RobustScaler()
train_standardized = scaler.fit_transform(japonica_training)
japonica_training_standard = pd.DataFrame(train_standardized, columns=japonica_training.columns, index=japonica_training.index)
test_standardized = scaler.fit_transform(japonica_validation)
japonica_validation_standard = pd.DataFrame(test_standardized, columns=japonica_validation.columns, index=japonica_validation.index)
japonica_training_features_X = japonica_training_standard.copy()
japonica_training_features_y = japonica_training_standard.copy()
japonica_validation_features_X = japonica_validation_standard.copy()
japonica_validation_features_y = japonica_validation_standard.copy()

In [5]:
japonica_training_features_X_chunks = []
japonica_training_features_y_chunks = []
japonica_validation_features_X_chunks = []
japonica_validation_features_y_chunks = []
for i in range(0, len(japonica_training_features_X) - 5):
    japonica_training_features_X_chunks.append(np.array(japonica_training_features_X.iloc[i:i+6].values, dtype=np.float64))
japonica_training_features_X_chunks = np.array(japonica_training_features_X_chunks, dtype=np.float64)
for i in range(0, len(japonica_training_features_y) - 5):
    japonica_training_features_y_chunks.append(np.array(japonica_training_features_y.iloc[i:i+6].values, dtype=np.float64))
japonica_training_features_y_chunks = np.array(japonica_training_features_y_chunks, dtype=np.float64)
for i in range(0, len(japonica_validation_features_X) - 5):
    japonica_validation_features_X_chunks.append(np.array(japonica_validation_features_X.iloc[i:i+6].values, dtype=np.float64))
japonica_validation_features_X_chunks = np.array(japonica_validation_features_X_chunks, dtype=np.float64)
for i in range(0, len(japonica_validation_features_y) - 5):
    japonica_validation_features_y_chunks.append(np.array(japonica_validation_features_y.iloc[i:i+6].values, dtype=np.float64))
japonica_validation_features_y_chunks = np.array(japonica_validation_features_y_chunks, dtype=np.float64)
japonica_training_features_X_chunks_crop = japonica_training_features_X_chunks[:-6]
japonica_training_features_y_chunks_crop = japonica_training_features_y_chunks[6:]
japonica_validation_features_X_chunks_crop = japonica_validation_features_X_chunks[:-6]
japonica_validation_features_y_chunks_crop = japonica_validation_features_y_chunks[6:]

(10645, 6, 7)
(10645, 6, 7)
(10645, 6, 7)
(10645, 6, 7)


In [6]:
huber_loss = Huber(delta=1.0)
model = Sequential()
model.add(LSTM(128, input_shape=(6, 7), return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(7, activation='linear')))
model.summary()
model.compile(optimizer='adam', loss=huber_loss, metrics=['mse','mae'])
checkpoint = ModelCheckpoint(model_name + '_best.keras', monitor='val_loss', save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
hist = model.fit(japonica_training_features_X_chunks_crop, japonica_training_features_y_chunks_crop, epochs = 1000, batch_size = 6, validation_data=(japonica_validation_features_X_chunks_crop, japonica_validation_features_y_chunks_crop), callbacks=[checkpoint, early_stop, reduce_lr])
model.save(model_name+'.keras')

c:\Users\rladn\anaconda3\envs\do\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 6, 128)         │        69,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 6, 64)          │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 6, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 6, 7)           │           231 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,687 (514.40 KB)

 Trainable params: 131,687 (514.40 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.2876 - mae: 0.4713 - mse: 2.9193 - val_loss: 0.1885 - val_mae: 0.3280 - val_mse: 2.3387 - learning_rate: 0.0010
Epoch 2/1000
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.1910 - mae: 0.3250 - mse: 2.3791 - val_loss: 0.1812 - val_mae: 0.3073 - val_mse: 2.3226 - learning_rate: 0.0010
Epoch 3/1000
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.1843 - mae: 0.3112 - mse: 2.3705 - val_loss: 0.1807 - val_mae: 0.3066 - val_mse: 2.3228 - learning_rate: 0.0010
Epoch 4/1000
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.1793 - mae: 0.3034 - mse: 2.3073 - val_loss: 0.1805 - val_mae: 0.3043 - val_mse: 2.3203 - learning_rate: 0.0010
Epoch 5/1000
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.1730 - mae: 0.2926 - mse: 2.2569 - val_loss: 0.1742 - val_mae: 0.2903 - val_mse: 2.3022 - learning_rate: 0.0010
Epoch 6/1000
1775/1775 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - loss: 0.1789 - mae: 0.2986 - mse: 2.3479 - val